import library

In [22]:
import duckdb

setup duckdb, ducklake, dan httpfs

In [23]:
con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL ducklake; LOAD ducklake;")

setup koneksi minio

In [24]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'localhost:9000'
}

con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

setup directory path (baik local directory maupun minio)

In [25]:
# Path Metadata Catalog
catalogMetadata = 'catalog.ducklake'

# Path Folder dimana file dikelola oleh catalog ducklake
managedStoredData = 'data-lakehouse/managedStoreData/'

# Daftar file yang akan dikelola oleh catalog ducklake
minioObjectName = {
    'jakarta': 'staging/parquet/duckdb/JawaBarat/jakarta.parquet',
    'bekasi': 'staging/parquet/duckdb/JawaBarat/bekasi.parquet',
    'depok': 'staging/parquet/duckdb/JawaBarat/depok.parquet',
    'balikpapan': 'staging/parquet/duckdb/kalimantanTimur/balikpapan.parquet',
    'makassar': 'staging/parquet/duckdb/sulawesiSelatan/makassar.parquet',
    'palembang': 'staging/parquet/duckdb/sumateraSelatan/palembang.parquet'
}

setup api endpoint minio untuk `object` dan `folder pengelolaan data ducklake`

In [26]:
objectEndpointURL = {
    'jakarta': f"s3://{minioAuth['bucket']}/{minioObjectName['jakarta']}",
    'bekasi': f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}",
    'depok': f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}",
    'balikpapan': f"s3://{minioAuth['bucket']}/{minioObjectName['balikpapan']}",
    'makassar': f"s3://{minioAuth['bucket']}/{minioObjectName['makassar']}",
    'palembang': f"s3://{minioAuth['bucket']}/{minioObjectName['palembang']}"
}

dataYangDikelolaDucklake = f"s3://{minioAuth['bucket']}/{managedStoredData}"

buat catalog database di lokal dengan format ducklake dan set lokasi menyimpanan data yang akan di kelola di datalake

In [27]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS dataLakehouse (DATA_PATH '{dataYangDikelolaDucklake}');")
con.sql("USE dataLakehouse;")

In [28]:
con.sql("CREATE SCHEMA IF NOT EXISTS staging;")

In [29]:
loadJakarta = f"""
        CREATE OR REPLACE TABLE staging.dataJakarta AS 
        SELECT * FROM read_parquet('{objectEndpointURL['jakarta']}');
    """
loadBekasi = f"""
        CREATE OR REPLACE TABLE staging.dataBekasi AS 
        SELECT * FROM read_parquet('{objectEndpointURL['bekasi']}');
    """
loadDepok = f"""
        CREATE OR REPLACE TABLE staging.dataDepok AS 
        SELECT * FROM read_parquet('{objectEndpointURL['depok']}');
    """
loadBalikpapan = f"""
        CREATE OR REPLACE TABLE staging.dataBalikpapan AS 
        SELECT * FROM read_parquet('{objectEndpointURL['balikpapan']}');
    """
loadMakassar = f"""
        CREATE OR REPLACE TABLE staging.dataMakassar AS 
        SELECT * FROM read_parquet('{objectEndpointURL['makassar']}');
    """
loadPalembang = f"""
        CREATE OR REPLACE TABLE staging.dataPalembang AS 
        SELECT * FROM read_parquet('{objectEndpointURL['palembang']}');
    """

In [30]:
try:
    con.sql(loadJakarta)
    con.sql(loadBekasi)
    con.sql(loadDepok)
    con.sql(loadBalikpapan)
    con.sql(loadMakassar)
    con.sql(loadPalembang)
    print(f"✅ Semua tabel staging terdaftar. (Source: MinIO, Managed: Lokal)")

except Exception as e:
    print(f"❌ Gagal memuat tabel | Error: {e}")

✅ Semua tabel staging terdaftar. (Source: MinIO, Managed: Lokal)


In [31]:
con.close()